# Ground State and Excited State of H2 Molecule using VQE and VQD
## Introduction

Quantum computing holds the promise of revolutionizing many fields, and one of the most exciting applications is in computational chemistry. Traditional methods for simulating molecular systems become computationally intractable as the size of the system increases. Quantum computers offer a potential solution to this problem by exploiting the quantum properties of matter to efficiently simulate molecular behavior.

In this notebook, we will employ two quantum algorithms, the Variational Quantum Eigensolver (VQE) and the Variational Quantum Deflation (VQD), to find the ground state and excited state of the $H_2$ molecule.

In [1]:
import functools 

import pennylane as qml
from pennylane import numpy as np
import optax
import jax

jax.config.update("jax_platform_name", "cpu")
jax.config.update('jax_enable_x64', True)

h2_dataset = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
h2 = h2_dataset[0]
H, qubits = h2.hamiltonian, len(h2.hamiltonian.wires)

In [2]:
h2.hf_state

tensor([1, 1, 0, 0], dtype=int64, requires_grad=True)

## VQE
The VQE needs the following:
- An Ansatz
- Loss function

In [3]:
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.22250914236600539) [Z2]
+ (-0.22250914236600539) [Z3]
+ (-0.09963387941370971) [I0]
+ (0.17110545123720225) [Z1]
+ (0.17110545123720233) [Z0]
+ (0.12051027989546245) [Z0 Z2]
+ (0.12051027989546245) [Z1 Z3]
+ (0.16584090244119712) [Z0 Z3]
+ (0.16584090244119712) [Z1 Z2]
+ (0.16859349595532533) [Z0 Z1]
+ (0.1743207725924201) [Z2 Z3]
+ (-0.04533062254573469) [Y0 Y1 X2 X3]
+ (-0.04533062254573469) [X0 X1 Y2 Y3]
+ (0.04533062254573469) [Y0 X1 X2 Y3]
+ (0.04533062254573469) [X0 Y1 Y2 X3]


## Groudtruth
Let's look at some of the eperical measured value
- Ground state energy:
  - $H$ atom: $E_1=-13.6eV$
  - $H_2$ molecule: $-1.136*27.21 Ha=-30.91 eV$
- 1st level excitation energy for $H$ atom: $E_2=\frac{-13.6}{4}=-3.4eV$
- The energy to transition from $E_1$ to $E_2$ for $H$ atom: $10.2eV$

In [4]:
def hatree_energy_to_ev(hatree: float):
    return hatree*27.2107

In [5]:
def ev_energy_to_hatree(ev: float):
    return ev/27.2107

## Begin training
Let's set some expectation for the optimization process. Thankfully, $H_2$ is well studied and we have all we need in the `dataset` library to know the ground truth

### Ansatz

Before any run, we can assume that the Jordan Wigner representation `[1 1 0 0]` has the lowest energy. Let's calculate that

In [6]:
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit_expected():
    qml.BasisState(h2.hf_state, wires = range(qubits))
    for op in h2.vqe_gates:
        qml.apply(op)
    return qml.probs(), qml.state(), qml.expval(H)

Since the Hamiltonian doesn't have a canonical representation, we have to create a work around

The idea herer is, for `Z(1)`, we convert into `I(0) @ Z(1) @ I(1) @ I(2)`, and do the same for every Hamiltonian operator

In [7]:
print(f"HF state: {h2.hf_state}")
prob, state, expval = circuit_expected()
print(f"Ground state energy H_2: {expval}")

HF state: [1 1 0 0]
Ground state energy H_2: -1.1363765762751892


In [8]:
@qml.qnode(dev)
def circuit_reverse(state):
    qml.AmplitudeEmbedding(state,wires=range(4))
    return qml.state()
    
circuit_reverse(state)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.13619566+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99068196+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j], requires_grad=True)

In [9]:
hatree_energy_to_ev(expval)

tensor(-30.9216021, requires_grad=True)

Taking the superposition with themselves and the higher/lower energy level (excite/de-excite). Note that in `h2.vqe_gates` we already have the value for $\theta$

In [10]:
print(qml.draw(circuit_expected)())

0: ─╭|Ψ⟩─╭G²(0.27)─┤  Probs  State ╭<𝓗>
1: ─├|Ψ⟩─├G²(0.27)─┤  Probs  State ├<𝓗>
2: ─├|Ψ⟩─├G²(0.27)─┤  Probs  State ├<𝓗>
3: ─╰|Ψ⟩─╰G²(0.27)─┤  Probs  State ╰<𝓗>


We would define the same circuit but without the $\theta$. Given 2 $H$ and 4 qubits, after a double excitation, the HF is the superposition of the states
$$\alpha\ket{1100}+\beta\ket{0011}:=\cos(\theta)\ket{1100}-\sin(\theta)\ket{0011}$$

[comment]: # ($\alpha\ket{110000}+\beta\ket{001100}+\gamma\ket{000011}$ this is H3)


In [11]:
@qml.qnode(dev, diff_method="backprop")
def circuit(param):
    qml.BasisState(h2.hf_state, wires=range(qubits))
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.state(), qml.expval(H)

### Define the lost function
Remember that the lost function is the second ingredient. We use the first two equations in [this paper](https://www.nature.com/articles/s41524-023-00965-1)
\begin{align}
C_0\left( {{{\mathbf{\theta }}}} \right) &= \left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle \label{eq:loss_1} \tag{1} \\
C_1\left( {{{\mathbf{\theta }}}} \right) &= \left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {\hat H} \right|{\Psi}\left( {{{\mathbf{\theta }}}} \right)} \right\rangle + \beta \left| {\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {{\Psi}_0} \right.} \right\rangle } \right|^2 \label{eq:loss_2} \tag{2}
\end{align}

We can then define a lost function

At first sight, it might raises some eyebrow for someone who is from a ML background, because we define the loss function based on the predicted and the groundtruth. However we do not have any groundtruth value here. In this context, a loss function is just a function that we want to minimize.

Now we proceed to optimize the variational parameters

Note that in equation \eqref{eq:loss_2}, we have the term $\beta \left| {\left\langle {{\Psi}\left( {{{\mathbf{\theta }}}} \right)\left| {{\Psi}_0} \right.} \right\rangle } \right|^2$, which is not easy to compute directly in a quantum machine. To make everything pure quantum, we rely on a swap test as below

In [12]:
dev_swap = qml.device("default.qubit", wires=1+2*qubits)

@qml.qnode(dev_swap)
def dot_product_circuit(psi, phi):
    """
    If psi and phi are orthogonal (|⟨psi|phi⟩|^2 = 1) then the probability that 0 is measured is 1/2 
    If the states are equal (|⟨psi|phi⟩|^2 = 1), then the probability that 0 is measured is 1.
    The measurement on the 0th wire, or 1st qubit is 0.5+0.5(|⟨psi|phi⟩|^2)
    """
      
    qml.AmplitudeEmbedding(features=psi, wires=range(1,qubits+1))
    qml.AmplitudeEmbedding(features=phi, wires=range(qubits+1,1+2*qubits))
    qml.Hadamard(0)
    for i in range(1, qubits+1):
        qml.CSWAP([0,i,i+qubits])
    qml.Hadamard(0)
    return qml.probs(0)

In [13]:
case_1_left, case_1_right = np.zeros(16), np.zeros(16)
case_1_left[12] = 1
case_1_right[12] = 1
assert np.all(np.isclose(dot_product_circuit(case_1_left,case_1_right), [1,0]))

case_2_left, case_2_right = np.zeros(16), np.zeros(16)
case_2_left[12] = 1/np.sqrt(2)
case_2_left[13] = 1/np.sqrt(2)
case_2_right[14] = 1/np.sqrt(2)
case_2_right[15] = 1/np.sqrt(2)
assert np.all(np.isclose(dot_product_circuit(case_2_left, case_2_right), [0.5,0.5]))

In [14]:
# np.real makes sense?
def loss_fn_1(theta):
    """
    Pure expectation value
    """
    _, expval = circuit(theta)
    return expval

def loss_fn_2(theta, theta_0, beta):
    """
    Expectation value
    Depends on the molecule, beta must be large enough to jump over the gap
    """
    state, expval = circuit(theta)
    state_0, _ = circuit(theta_0)
    dot_prod = (dot_product_circuit(state, state_0)[0] - 0.5)/0.5    
    return expval + beta*dot_prod

In [15]:
def optimize(loss_f, **kwargs):
    theta = np.array(0.)

    # store the values of the cost function
    energy = [loss_fn_1(theta)]
    conv_tol = 1e-6
    max_iterations = 100
    opt = optax.sgd(learning_rate=0.4)
    
    # store the values of the circuit parameter
    angle = [theta]
    
    opt_state = opt.init(theta)
    
    for n in range(max_iterations):
        gradient = jax.grad(loss_f)(theta, **kwargs)
        updates, opt_state = opt.update(gradient, opt_state)
        theta = optax.apply_updates(theta, updates)
        
        angle.append(theta)
        energy.append(loss_fn_1(theta))
    
        conv = np.abs(energy[-1] - energy[-2])
    
        if n % 5 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
    
        if conv <= conv_tol:
            break
    return angle[-1], energy[-1]

### Run the ground state optimization

In [16]:
ground_state_theta, ground_state_energy = optimize(loss_fn_1)

C:\code\vqeac\.venv\lib\site-packages\pennylane\math\utils.py:227: UserWarning: Contains tensors of types {'jax', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
C:\code\vqeac\.venv\lib\site-packages\jax\_src\lax\lax.py:2740: ComplexWarning: Casting complex values to real discards the imaginary part
  x_bar = _convert_element_type(x_bar, x.aval.dtype, x.aval.weak_type)
C:\code\vqeac\.venv\lib\site-packages\pennylane\math\utils.py:227: UserWarning: Contains tensors of types {'jax', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Step = 0,  Energy = -1.12772109 Ha
Step = 5,  Energy = -1.13706903 Ha
Step = 10,  Energy = -1.13725940 Ha


### Run the 1st excited state optimization

Next we are going to choose the value for $\beta$, it has to be big enough to facilitate a jump

In [17]:
beta = 5

In [18]:
first_excite_theta, first_excite_energy = optimize(loss_fn_2, theta_0=ground_state_theta, beta=beta)

C:\code\vqeac\.venv\lib\site-packages\pennylane\math\utils.py:227: UserWarning: Contains tensors of types {'jax', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Step = 0,  Energy = -1.08055668 Ha
Step = 5,  Energy = 0.42901254 Ha
Step = 10,  Energy = 0.47841911 Ha


In [19]:
hatree_energy_to_ev(ground_state_energy), hatree_energy_to_ev(first_excite_energy)

(Array(-30.94570863, dtype=float64), Array(13.01809466, dtype=float64))